In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_scheduler
import torch
from torch.nn.functional import softmax
import datasets
from datasets import load_dataset, Dataset
import random
from torch.utils.data import DataLoader, Subset
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchmetrics import F1Score
import pandas as pd
import numpy as np
pd.options.mode.copy_on_write = True

e:\anaconda3\envs\ece1786\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [6]:
reverse_mapping_3 = {
    3.5: 0, 4.0: 0,
    4.5: 1, 5.0: 1,
    5.5: 2, 6.0: 2,
    6.5: 3, 7.0: 3,
    7.5: 4, 8.0: 4,
    8.5: 5, 9.0: 5
}

class_mapping = {
    0: "3.5 - 4.0",
    1: "4.5 - 5.0",
    2: "5.5 - 6.0",
    3: "6.5 - 7.0",
    4: "7.5 - 8.0",
    5: "8.5 - 9.0",
}

In [2]:
categories = ['task_achievement', 
              'grammatical',
              # 'coherent',  # not available
              # 'lexical'  # not available
              ]
models = []

for category in categories:
    model = AutoModelForSequenceClassification.from_pretrained("mrm8488/deberta-v3-ft-financial-news-sentiment-analysis",num_labels=6, ignore_mismatched_sizes=True)
    # Load the saved state_dict into the model
    with open(f'models/{category}.pt', "rb") as f:
        model.load_state_dict(torch.load(f))

    # Move model to the device (GPU if available)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model.to(device)
    models.append(model)

tokenizer = AutoTokenizer.from_pretrained("mrm8488/deberta-v3-ft-financial-news-sentiment-analysis")

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at mrm8488/deberta-v3-ft-financial-news-sentiment-analysis and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([6]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\20838\AppData\Local\Temp\ipykernel_7868\2412511595.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the 

In [26]:
prompt = """ Some people feel that with the rise of artificial intelligence, computers and robots will take over the roles of teachers. To what extent do you agree or disagree with this statement? 
Give reasons for your answer and include any relevant examples from your knowledge or experience. 
You should write at least 250 words. """

essay = """ With ever increasing technological advances, computers and robots are replacing human roles in different areas of society. This trend can also be seen in education, where interactive programs can enhance the educational experience for children and young adults. Whether, however, this revolution can also take over the role of the teacher completely is debatable, and I oppose this idea as it is unlikely to serve students well. 
The roles of computers and robots can be seen in many areas of the workplace. Classic examples are car factories, where a lot of the repetitive precision jobs done on assembly lines have been performed by robots for many years, and medicine, where diagnosis, and treatment, including operations, have also been assisted by computers for a long time. According to the media, it won't also be long until we have cars that are driven automatically. 
It has long been discussed whether robots and computers can do this in education. It is well known that the complexity of programs can now adapt to so many situations that something can already be set up that has the required knowledge of the teacher, along with the ability to predict and answer all questions that might be asked by students. In fact, due to the nature of computers, the knowledge levels can far exceed a teacher's and have more breadth, as a computer can have equal knowledge in all the subjects that are taught in school, as opposed to a single teacher's specialisation. It seems very likely, therefore, that computers and robots should be able to deliver the lessons that teachers can, including various ways of differentiating and presenting materials to suit varying abilities and ages of students. 
Where I am not convinced is in the pastoral role of teachers. Part of teaching is managing behaviour and showing empathy with students, so that they feel cared for and important. Even if a robot or computer can be programmed to imitate these actions, students will likely respond in a different way when they know an interaction is part of an algorithm rather than based on human emotion. 
Therefore, although I feel that computers should be able to perform a lot of the roles of teachers in the future, they should be used as educational tools to assist teachers and not to replace them. In this way, students would receive the benefits of both ways of instruction. 
"""
comment = """Will the comment affect the overall score?"""

example_input1 = f"""Prompt: {prompt}
Essay: {essay}
Comment: {comment}
"""

In [27]:
for i, model in enumerate(models):
    inputs = tokenizer(example_input1, return_tensors="pt").to(device)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = logits.argmax(dim=1).item()

    print(f"{categories[i]}: {class_mapping[predicted_class]}")

task_achievement: 5.5 - 6.0
grammatical: 6.5 - 7.0
